In [ ]:
# !pip install deeptables[gpu]


In [ ]:
from deeptables.models.deeptable import DeepTable, ModelConfig
from deeptables.models.deepnets import WideDeep, DeepFM, xDeepFM, DCN, PNN, AutoInt, AFM, FGCNN

from deeptables.datasets import dsutils
from deeptables.utils.batch_trainer import BatchTrainer


from google.colab import files

m = [WideDeep, DeepFM, xDeepFM, DCN, PNN, AutoInt, AFM, FGCNN]
for i in m:
  print(i)
data = dsutils.load_adult()
data.head(10)



['linear', 'dnn_nets']
['linear', 'fm_nets', 'dnn_nets']
['linear', 'cin_nets', 'dnn_nets']
['dcn_nets']
['pnn_nets']
['autoint_nets']
['afm_nets']
['fgcnn_dnn_nets']
Base dir:/usr/local/lib/python3.6/dist-packages/deeptables/datasets


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [ ]:
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
from tqdm import tqdm
tqdm.pandas()
from catboost import CatBoostClassifier

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold
from sklearn import metrics
from lightgbm import LGBMClassifier
from IPython.display import FileLink

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train_file = '/content/gdrive/My Drive/train.csv'
test_file = '/content/gdrive/My Drive/test.csv'
filename = 'result.csv'

In [ ]:
df, tdf = pd.read_csv(train_file), pd.read_csv(test_file)

In [ ]:
df.shape, tdf.shape, df.id.nunique(), tdf.id.nunique()

((381109, 12), (127037, 11), 381109, 127037)

In [ ]:
cat_cols = ['Gender', 'Driving_License', 'Region_Code',
        'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage']
reg_cols = [ 'Age', 'Annual_Premium','Vintage']
useless_cols = ['id']
target_col = ['Response']
spl_cat_col = ['Policy_Sales_Channel']

In [ ]:
X = df[df.columns[~(df.columns.isin(['id']))].tolist()]

y = df[target_col[0]].values
print(type(X), type(y))

<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>


In [ ]:
X.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [ ]:

conf1 = ModelConfig(nets=WideDeep, metrics=['AUC'], auto_categorize=True,
                    apply_class_weight=True, 
                    auto_discrete=True)
                # fixed_embedding_dim=False)# categorical_columns=cat_cols + spl_cat_col,
                
conf2 = ModelConfig(nets=WideDeep, metrics=['AUC'], auto_categorize=True,apply_class_weight=True)

bt = BatchTrainer(X, 'Response', dt_nets=[['linear', 'dnn_nets'], ['dnn_nets','fm_nets']], eval_metrics=['AUC'], dt_epochs=100, dt_config=conf1, cross_validation=True, num_folds=5, n_jobs=1)
# dt = DeepTable(config=conf)
# model, history = dt.fit(X, y, epochs=100)
ms = bt.start()
  

  
  

2 class detected, {0, 1}, so inferred as a [binary classification] task
Start training DT model.['linear', 'dnn_nets']
train metrics:['AUC']
eval metrics:['AUC']
Fitting model...
Start cross validation
2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features cost:0.9172763824462891
Imputation cost:0.6908218860626221
Categorical encoding cost:1.0928106307983398
Discretization cost:0.41994237899780273
fit_transform cost:3.200289726257324
transform X_eval
transform_X cost:6.643517255783081
Iterators:StratifiedKFold(n_splits=5, random_state=9527, shuffle=True)
Calc classes weight.
Examples:
Total:304887
class 0:0.5699014172439446
class 1:4.076465397368702
Injected a callback [EarlyStopping]. monitor:val_AUC, patience:1, mode:max


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

In [ ]:

# X_test = tdf[tdf.columns[~(tdf.columns.isin(['id', target_col[0]]))].tolist()]
# preds = dt.predict(X_test)
# print(sum([1 for i in preds if i == 1]) / len(preds), len(preds))
# result = pd.DataFrame({'id': tdf['id'], 'Response': preds.ravel()})
# result.to_csv(filename, index=False)

In [ ]:
# sum([1 for i in y if i == 1]) / len(y), len(y)

In [ ]:

# files.download(filename)

In [ ]:
# X.shape, X_test.shape

In [ ]:
# !ls

In [ ]:
ms.leaderboard(top=100)

In [ ]:
# import pickle as pkl
# pkl.dump(preds, open('result.pkl', 'wb'))

In [ ]:
# files.download('result.pkl')

In [ ]:
# preds.keys()